In [36]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb


from path_setup import setup_paths
setup_paths()

In [37]:
from dataset_transformer import BaseDatasetTransform
from base_model_train import BaseModelTrain
from model_evaluator import ModelEvaluator

In [38]:
df = pd.read_csv('../datasets/Music_genre_Classification.csv')

In [39]:
df.shape

(17996, 17)

In [40]:
bdf = BaseDatasetTransform(df,target='Class')

In [41]:
df,categorical_features = bdf.fit_transform()


В колонке Class нет пропущенных значений
-------------------------------------------
В наборе данных есть пропущенные значения:
Колонки с пропущенными значениями:
Popularity           428
key                 2014
instrumentalness    4377
dtype: int64
Все пропущенные значения заполнены.
-------------------------------------------
Информация о колонках в датасете

Категориальные колонки:
Index(['Artist Name', 'Track Name'], dtype='object')

Числовые колонки:
Index(['Popularity', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'duration_in min/ms', 'time_signature', 'Class'],
      dtype='object')


In [42]:
X_train, X_test, y_train, y_test  = bdf.get_train_test_split()

Количество значений целевой переменной по категориям
Class
10    4949
6     2587
9     2524
8     1854
5     1447
1     1373
2     1272
0      625
7      576
3      402
4      387
Name: count, dtype: int64
Следует ли выполнить стратифицированное раздеение на обучающую и тестовую выборку? y/n
Разделение датасета выполнено успешно


In [43]:
from sklearn.utils.class_weight import compute_class_weight 
#is_unbalance=True

class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(enumerate(class_weights))

class_counts = y_train.value_counts()


lgbm_model = lgb.LGBMClassifier(verbose=-1,class_weight=class_weights_dict)

columns_to_delete = []  

unique_classes = len(set(y_train))

model_train = BaseModelTrain(
    model=lgbm_model,
    columns_to_delete=columns_to_delete,
    categorical_features=categorical_features,
    boosting_type='gbdt',
    objective='multiclass',  
    num_class=unique_classes, 
    metric='multi_error' 
)

In [44]:
metrics = ['accuracy', 'precision', 'recall', 'f1_macro']
evaluator = ModelEvaluator(model_trainer=model_train, metrics=metrics)

evaluator.fit(X_train, y_train)

ModelEvaluator(metrics=['accuracy', 'precision', 'recall', 'f1_macro'],
               model_trainer=BaseModelTrain(categorical_features=['Artist Name',
                                                                  'Track Name'],
                                            columns_to_delete=[],
                                            model=LGBMClassifier(class_weight={0: 2.6145703611457036,
                                                                               1: 1.1916564185034528,
                                                                               2: 1.2867211440245148,
                                                                               3: 4.075380135878357,
                                                                               4: 4.225763166722576,
                                                                               5: 1.1304855065960693,
                                                                               6: 0.6323477737061393,
                                                                               7: 2.841642228739003,
                                                                               8: 0.8822664238688892,
                                                                               9: 0.6480938416422287,
                                                                               10: 0.3305952131009868},
                                                                 metric='multi_error',
                                                                 num_class=11,
                                                                 objective='multiclass',
                                                                 verbose=-1)))

In [45]:
evaluator.evaluate_to_dataframe(X_test, y_test)

,Metric,Score
0,accuracy,0.540841
1,precision,0.604305
2,recall,0.632169
3,f1,0.616112
